# 第二章：字符串和文本

## 2.1 使用多个界定符分割字符串

问题：需要将一个字符串分割为多个字段，但是分隔符（还有周围的空格）并不是固定的

解决方案：string对象的split方法只适应于非常简单的字符串分割情形，它并不允许有多个分隔符或者分隔符周围不确定的空格
，当你需要更加灵活的切割字符串的时候，最好使用re.split()方法

In [2]:
line = 'asdf fjdk;  afed, fjed,asdf,   foo'
import re
re.split(r'[;,\s]\s*', line)

['asdf', 'fjdk', 'afed', 'fjed', 'asdf', 'foo']

分隔符可以是逗号，分好或者空格，并且后面紧跟着任意个的空格

当你使用re.split函数时候，需要特别注意的是正则表达式中是否包含一个括号捕获分组，如果使用捕获分组，那么被匹配的文本也将出现结果列表中，比如

In [3]:
fields = re.split(r'(;|,|\s)\s*', line)
fields

['asdf', ' ', 'fjdk', ';', 'afed', ',', 'fjed', ',', 'asdf', ',', 'foo']

获取分割字符在某些情况下也是有用的，比如

In [4]:
values = fields[::2]
delimiters = fields[1::2] + ['']
values,delimiters

(['asdf', 'fjdk', 'afed', 'fjed', 'asdf', 'foo'],
 [' ', ';', ',', ',', ',', ''])

In [5]:
''.join(v+d for v, d in zip(values, delimiters))

'asdf fjdk;afed,fjed,asdf,foo'

如果你不想保留分割字符串到结果列表去，但仍然需要使用括号来分组正则表达式的话，确保你的分组是非捕获分组
形如，(?:......)，比如：

In [6]:
re.split(r'(?:,|;|\s)\s*', line) #这个跟In2的区别是？

['asdf', 'fjdk', 'afed', 'fjed', 'asdf', 'foo']

## 2.2 字符串开头或结尾匹配

问题：你需要指定的文本模式去检查字符串的开头或者结尾，比如，文件后缀，URL Scheme等

解决方案：简单方法是使用str.startswith() str.endswith()

In [1]:
filename = 'spam.txt'
filename.endswith('.txt')

True

In [2]:
url = 'http://www.python.org'
url.startswith('http:')

True

如果你想检查多种匹配可能，只需要讲所有的匹配项放到一个元组去，然后传递给startswith等

In [3]:
import os
filenames = os.listdir('.') # !
filenames

['.ipynb_checkpoints', 'CookBook_1.ipynb', 'CookBook_2.ipynb', 'somefile.txt']

In [4]:
[name for name in filenames if name.endswith(('ipynb','txt'))]

['CookBook_1.ipynb', 'CookBook_2.ipynb', 'somefile.txt']

In [5]:
any(name.endswith('.txt') for name in filenames)

True

In [6]:
from urllib.request import urlopen

def read_data(name):
    if name.startswith(('http', 'https', 'ftp:')):
        return urlopen(name).read()
    else:
        with open(name) as f:
            return f.read()
choices = ['http:', 'ftp:']

In [7]:
url.startswith(choices)

TypeError: startswith first arg must be str or a tuple of str, not list

In [8]:
#类似的操作也可以使用切片来实现；还能使用正则表达式实现
#切片省略
#正则表达式

import re
url = 'http://www.python.org'
re.match('http:|https:|ftp:', url)

<re.Match object; span=(0, 5), match='http:'>

## 2.3 用shell通配符匹配字符串

问题：你想使用linux shell中的常用的通配符，比如，*.py,Dat[0-9]*.csv等去匹配文本字符串

解决方法：fnmatch模块提供两个函数 fnmatch和fnmatchcase，可以用来实现这样的匹配，比如

In [9]:
from fnmatch import fnmatch, fnmatchcase

fnmatch('foo.txt', '*.txt'), fnmatch('too.txt', '?oo.txt'),fnmatch('Dat45.csv', 'Dat[0-9]*')

(True, True, True)

In [10]:
names = ['Dat1.csv', 'Dat2.csv', 'config.ini', 'foo.py']
[name for name in names if fnmatch(name, 'Dat*.csv')]

['Dat1.csv', 'Dat2.csv']

fnmatch对底层操作系统大小写敏感，如果对这个区别很在意，可以使用fnmatchcase来代替，它完全使用你的模式大小写匹配，比如

In [14]:
fnmatch('foo.txt', '*.TXT')

True

In [15]:
fnmatchcase('foo.txt','*.TXT')

False

这两个函数通常会被忽略的一个特性是在处理非文件名的字符串时，它们是有用的，比如，一个街道
数据

In [16]:
address = [
    '5412 N CLARK ST',
    '1060 W ADDISON ST',
    '1039 W GRANVILLE AVE',
    '2122 N CLARK ST',
    '4802 N BROADWAY'
]

In [17]:
from fnmatch import fnmatchcase

[addr for addr in address if fnmatchcase(addr, '* ST')]

['5412 N CLARK ST', '1060 W ADDISON ST', '2122 N CLARK ST']

In [18]:
[addr for addr in address if fnmatchcase(addr, '54[0-9][0-9] *CLARK*')]

['5412 N CLARK ST']

2.4 字符串匹配和搜索

问题：你想匹配或者搜索特定模式的文本

In [19]:
text = 'yeah, but no, but yeah, but no, but yeah'
text.find('no')

10

对于复杂的匹配需要使用正则表达式和re模块，为了解释正则表达式的基本原理，假设你想匹配数字
格式的日期字符串，比如11/22/2012

In [20]:
text1 = '11/27/2012'
text2 = 'Nov 27, 2012'

import re

if re.match(r'\d+/\d+/\d+',text1):
    print('yes')
else:
    print('no')

if re.match(r'\d+/\d+/\d+',text2):
    print('yes')
else:
    print('no')

yes
no


如果你想使用同一个模式做多次匹配，你应该先将模式字符串预编译为模式对象，比如

In [21]:
datepat = re.compile(r'\d+/\d+/\d+')

if datepat.match(text1):
    print('yes')
else:
    print('no')
    
if datepat.match(text2):
    print('yes')
else:
    print('no')

yes
no


match总是从字符串开始去匹配，如果你想查找字符串任意部分的模式出现位置，使用findall方法

In [22]:
text = 'Todau is 11/27/2012, PyCon starts 3/13/2013.'
datepat.findall(text)

['11/27/2012', '3/13/2013']

在定义正则式的时候，通常会利用括号去捕获分组，比如：

In [27]:
datepat = re.compile(r'(\d+)/(\d+)/(\d+)')

m = datepat.match('11/28/2012')
m,m.group(0), m.group(1), m.group(2),m.group(3), m.groups()

(<re.Match object; span=(0, 10), match='11/28/2012'>,
 '11/28/2012',
 '11',
 '28',
 '2012',
 ('11', '28', '2012'))

In [30]:
datepat.findall(text)

[('11', '27', '2012'), ('3', '13', '2013')]

In [31]:
for month, day, year in datepat.findall(text):
    print('{}-{}-{}'.format(year, month, day))

2012-11-27
2013-3-13


findall方法会搜索文本并以列表形式返回所有的匹配，如果想以迭代凡是返回匹配，可以使用
finditer方法来代替，比如

In [32]:
for m in datepat.finditer(text):
    print(m.groups())

('11', '27', '2012')
('3', '13', '2013')


In [33]:
m = datepat.match('11/27/2012abcdefg')
m

<re.Match object; span=(0, 10), match='11/27/2012'>

In [34]:
m.group()

'11/27/2012'

In [35]:
datepat = re.compile(r'(\d+)/(\d+)/(\d+)$')
datepat.match('11/27/2012abcedfg')

In [36]:
datepat.match('11/27/2012')

<re.Match object; span=(0, 10), match='11/27/2012'>

## 2.5 字符串搜索和替换

问题：想在字符串中搜索和匹配指定的文本模式

解决方法：str.replace;re.sub()

In [38]:
text = 'yeah, but no, but yeah, but no,but yeah'
text.replace('yeah', 'yep')

'yep, but no, but yep, but no,but yep'

In [46]:
text = 'Today is 11/27/2012, PyCon starts 3/13/2013.'
import re
re.sub(r'(\d+)/(\d+)/(\d+)',r'\3-\1-\2', text) #反斜杠数字比如\3指向前面模式的捕获组号

'Today is 2012-11-27, PyCon starts 2013-3-13.'

对于更复杂的替换，可以传递一个替换回调函数来代替，比如

In [49]:
from calendar import month_abbr

datepat = re.compile(r'(\d+)/(\d+)/(\d+)')

def change_date(m):
    mon_name = month_abbr[int(m.group(1))]
    return '{} {} {}'.format(m.group(2), mon_name, m.group(3))

datepat.sub(change_date, text)


'Today is 27 Nov 2012, PyCon starts 13 Mar 2013.'

In [50]:
newtext, n = datepat.subn(r'\3-\1-\2', text)
newtext, n

('Today is 2012-11-27, PyCon starts 2013-3-13.', 2)

## 2.6 字符串忽略大小写的搜索替换

问题：你需要以忽略大小写的方式搜索与替换文本字符串

解决方法：re模块时给这些才啊哦做提供re.IGNORECASE标志参数

## 2.7 最短匹配模式（非贪心）

问题：你正在试用正则表达式匹配某个文本模式，但是它找到的是模式的最长可能匹配，而你想修改
它变成查找最短的可能匹配

解决方案：这个问题一般出现在需要匹配一对分隔符之间的文本的时候（比如引号包含的字符串）

In [52]:
str_pat = re.compile(r'\"(.*)\"') # 匹配被双引号包含的文本
text1 = 'Computer says "no."'
str_pat.findall(text1)

['no.']

In [54]:
text2 = 'Computer says "no." Phone says "yes."'
str_pat.findall(text2)

['no." Phone says "yes.']

正则表达式中*操作符是贪婪的，因为匹配操作会查找最长的可能匹配，第二个例子中返回结果
并不是我们想要的，为了修正这个问题，可以在模式中*操作符后面加上？修饰符

In [55]:
str_pat = re.compile(r'\"(.*?)\"')
str_pat.findall(text2)

['no.', 'yes.']

可以在*或者+这样的操作符后面添加一个？，可以强制匹配寻找最短的可能匹配

## 2.8 多行匹配模式

问题：你正试使用正则表达式去匹配一大块的文本，而你需要跨越多行去匹配

解决方案：这个问题很典型的出现当你用.去匹配任意字符的时候，忘记了.不能匹配换行符的事实
，比如，假设你想试着去匹配C语言分割的注释

In [57]:
comment = re.compile(r'/\*(.*?)\*/')
text1 = '/* this is a comment */'
text2 = '''/* this is a 
multiline comment */'''
comment.findall(text1)

[' this is a comment ']

In [58]:
comment.findall(text2)

[]

In [59]:
comment = re.compile(r'/\*((?:.|\n)*?)\*/')
comment.findall(text2)

[' this is a \nmultiline comment ']

(?:.|\n)指定了一个非捕获组，也就是定义了一个仅仅用来做匹配，而不能通过单独捕获或者编号的组

更为简单使用一个标志参数 re.DOTALL,他可以让正则表示中的点匹配包括换行符在内的任意字符

In [61]:
comment = re.compile(r'/\*(.*?)\*/',re.DOTALL)
comment.findall(text2)

[' this is a \nmultiline comment ']

### 2019.07.17